In [562]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [563]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

from market_value_predictor.preproc import reduce_number_of_classes

In [564]:
random.seed(42)

# Import data

In [565]:
df_train = pd.read_csv("../../raw_data/master_data_train.csv").drop(columns="Unnamed: 0")
df_test = pd.read_csv("../../raw_data/master_data_test.csv").drop(columns="Unnamed: 0")

In [566]:
df_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 101 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    fee_cleaned                    3132 non-null   float64
 1    age                            3132 non-null   int64  
 2    height_cm                      3132 non-null   int64  
 3    weight_kg                      3132 non-null   int64  
 4    nationality                    3132 non-null   object 
 5    club_name                      3113 non-null   object 
 6    league_name                    3113 non-null   object 
 7    league_rank                    3113 non-null   float64
 8    overall                        3132 non-null   int64  
 9    potential                      3132 non-null   int64  
 10   wage_eur                       3132 non-null   int64  
 11   player_positions               3132 non-null   object 
 12   preferred_foot                 3

In [567]:
numericals = ["int64", "float64"]

In [568]:
X = df_train.select_dtypes(numericals).drop(columns="fee_cleaned")

In [569]:
y = df_train.fee_cleaned

# Feature selection

In [570]:
correlations = df_train.select_dtypes(numericals).corr().fee_cleaned

In [571]:
correlations.sort_values(ascending=False)[1:11]

release_clause_eur          0.540972
potential                   0.459752
wage_eur                    0.445108
gk_handling                 0.386970
overall                     0.366983
international_reputation    0.331902
gk_positioning              0.310531
movement_reactions          0.308766
gk_kicking                  0.300016
gk_reflexes                 0.294949
Name: fee_cleaned, dtype: float64

In [572]:
top_5_corr_num_features = list(correlations.sort_values(ascending=False)[1:6].index)

In [573]:
X_selection = X[top_5_corr_num_features]

In [574]:
X_selection

,release_clause_eur,potential,wage_eur,gk_handling,overall
0,1500000.0,77,2000,NaN,63
1,3600000.0,83,6000,NaN,67
2,625000.0,78,650,NaN,58
3,27600000.0,81,60000,NaN,80
4,14800000.0,79,30000,NaN,74
...,...,...,...,...,...
3127,NaN,80,30000,NaN,76
3128,NaN,72,40000,NaN,70
3129,24200000.0,84,9000,NaN,75
3130,4900000.0,86,15000,NaN,68


# Train model 1: top 5 correlated numerical features

In [632]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [576]:
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", MinMaxScaler()),
    ("estimator", LinearRegression())
])
pipe

Pipeline(steps=[('imputer', SimpleImputer(fill_value=0, strategy='constant')),
                ('scaler', MinMaxScaler()), ('estimator', LinearRegression())])

In [577]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_selection, y, cv=5, scoring='r2').mean()

0.28870471180551427

# Feature engineering

In [578]:
from market_value_predictor.preproc import manual_encoding

In [579]:
df_train = manual_encoding(df_train, "player_tags")

In [580]:
df_train = manual_encoding(df_train, "player_positions")

In [581]:
df_train = manual_encoding(df_train, "player_traits")

In [582]:
df_train.select_dtypes(object)

,nationality,club_name,league_name,preferred_foot,work_rate,body_type,real_face,team_position
0,Italy,FC Sion,Swiss Super League,Left,High/Medium,Normal,No,LCM
1,France,Paris Saint-Germain,French Ligue 1,Right,Medium/Medium,Normal,No,RES
2,Ecuador,Club Atlético Talleres,Argentina Primera División,Left,Medium/Medium,Lean,No,SUB
3,Russia,FC Krasnodar,Russian Premier League,Right,High/Medium,Lean,No,ST
4,Serbia,Olympique de Marseille,French Ligue 1,Right,High/Low,Lean,Yes,SUB
...,...,...,...,...,...,...,...,...
3127,France,OGC Nice,French Ligue 1,Right,Medium/Low,Normal,No,SUB
3128,Norway,Rosenborg BK,Norwegian Eliteserien,Right,Medium/High,Normal,No,ST
3129,Brazil,Portimonense SC,Portuguese Liga ZON SAGRES,Left,Medium/Low,Normal,No,RW
3130,France,Olympique Lyonnais,French Ligue 1,Right,Medium/Medium,Normal,No,SUB


## Reduce number of classes on 'nationality'

In [583]:
nationality_df = pd.DataFrame(
    df_train.nationality.value_counts()).reset_index().rename(
        columns={
            "index": "nationality",
            "nationality": "count"
        })

In [584]:
temp_list = []
for i, elem in enumerate(list(nationality_df["count"])):
    if elem > 50:
        temp_list.append(list(nationality_df.nationality)[i])
    else:
        temp_list.append("other")

In [585]:
nationality_df["nationality_cleaned"] = temp_list

In [586]:
nationality_df = nationality_df.drop(columns="count")

In [587]:
df_train = df_train.merge(nationality_df, on="nationality", how="left").drop(columns="nationality")

## Reduce number of classes on 'league_name'

In [588]:
league_name_df = pd.DataFrame(
    df_train.league_name.value_counts()).reset_index().rename(
        columns={
            "index": "league_name",
            "league_name": "count"
        })

In [589]:
temp_list = []
for i, elem in enumerate(list(league_name_df["count"])):
    if elem > 100:
        temp_list.append(list(league_name_df.league_name)[i])
    else:
        temp_list.append("other")

In [590]:
league_name_df["league_name_cleaned"] = temp_list

In [591]:
league_name_df = league_name_df.drop(columns="count")

In [592]:
df_train = df_train.merge(league_name_df, on="league_name", how="left").drop(columns="league_name")

## Club_name

In [593]:
ohe = OneHotEncoder(sparse=False)

In [594]:
club_name_encoded = pd.DataFrame(ohe.fit_transform(df_train[["club_name"]]))

In [595]:
club_name_encoded.columns = ohe.get_feature_names_out()

In [596]:
club_name_encoded["fee_cleaned"] = df_train.fee_cleaned

In [597]:
club_name_encoded.corr().fee_cleaned.sort_values(ascending=False)[:20]

fee_cleaned                          1.000000
club_name_Athletic Club de Bilbao    0.131095
club_name_Borussia Dortmund          0.106269
club_name_Real Madrid                0.103005
club_name_FC Barcelona               0.091204
club_name_Juventus                   0.087745
club_name_Paris Saint-Germain        0.079938
club_name_Liverpool                  0.077838
club_name_SL Benfica                 0.076187
club_name_Chelsea                    0.074385
club_name_Manchester United          0.072174
club_name_Ajax                       0.068007
club_name_Leicester City             0.062929
club_name_Atlético Madrid            0.062028
club_name_FC Porto                   0.061783
club_name_Valencia CF                0.059865
club_name_Southampton                0.059597
club_name_Roma                       0.059025
club_name_Inter                      0.052526
club_name_Everton                    0.051069
Name: fee_cleaned, dtype: float64

In [598]:
### Drop club_name

In [599]:
df_train.drop(columns="club_name", inplace=True)

## Reduce number of classes on 'team_position'

In [600]:
attack = ["ST", "LS", "LW", "RS", "RW", "RF", "LF", "CF"]
mid = ["LCM", "RM", "CB", "CAM", "LM", "CM", "CDM", "RCM", "LCM", "RDM", "LDM", "RAM", "LAM"]
defense = ["RCB", "LCB", "CB", "RB", "LB", "RWB", "LWB"]
goal = ["GK"]
sub = ["SUB", "RES"]

In [601]:
df_train["position_cluster"] = df_train.team_position.map(lambda x: "attack" if x in attack else "mid"
                           if x in mid else "defense" if x in defense else "goal"
                           if x in goal else "sub" if x in sub else "nan")

In [602]:
np.unique(temp, return_counts=True)

(array(['attack', 'defense', 'goal', 'mid', 'nan', 'sub'], dtype=object),
 array([ 393,  504,  124,  666,   19, 1426]))

In [603]:
np.unique(list(df_train.team_position), return_counts=True)

(array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LAM', 'LB', 'LCB', 'LCM',
        'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM',
        'RDM', 'RES', 'RF', 'RM', 'RS', 'RW', 'RWB', 'ST', 'SUB', 'nan'],
       dtype='<U3'),
 array([  92,   12,   35,    4,   21,  124,    6,  110,  141,   98,   45,
           6,  102,   62,   52,    6,    8,  105,  132,   95,   59,  230,
           6,   93,   65,   49,   10,  149, 1196,   19]))

In [604]:
df_train.drop(columns="team_position", inplace=True)

# Drop nas from object columns

In [605]:
df_train = df_train.dropna(subset=list(df_train.select_dtypes(object).columns))

In [606]:
df_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3113 entries, 0 to 3131
Data columns (total 165 columns):
 #    Column                                  Non-Null Count  Dtype  
---   ------                                  --------------  -----  
 0    fee_cleaned                             3113 non-null   float64
 1    age                                     3113 non-null   int64  
 2    height_cm                               3113 non-null   int64  
 3    weight_kg                               3113 non-null   int64  
 4    league_rank                             3113 non-null   float64
 5    overall                                 3113 non-null   int64  
 6    potential                               3113 non-null   int64  
 7    wage_eur                                3113 non-null   int64  
 8    preferred_foot                          3113 non-null   object 
 9    international_reputation                3113 non-null   int64  
 10   weak_foot                               3113 n

In [607]:
list(df_train.columns)

['fee_cleaned',
 'age',
 'height_cm',
 'weight_kg',
 'league_rank',
 'overall',
 'potential',
 'wage_eur',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'team_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_pen

# Train model 2: After feature engineering

In [608]:
all_cats = list(df_train.select_dtypes(object).columns)

In [609]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

all_numerics = list(df_train.select_dtypes(include=numerics).columns)

all_numerics.remove("fee_cleaned")

In [610]:
encoded_columns = [elem for elem in all_numerics if "player_tags_" in elem] + [
    elem for elem in all_numerics if "player_positions_" in elem
] + [elem for elem in all_numerics if "player_traits_" in elem]

In [611]:
all_numerics_wo_encoded = []
for elem in all_numerics:
    if elem not in encoded_columns:
        all_numerics_wo_encoded.append(elem)

In [612]:
numericals_zero_impute = [
    "gk_diving", "gk_handling", "gk_kicking", "gk_reflexes", "gk_speed",
    "gk_positioning", "release_clause_eur"
]

In [613]:
numericals_mean_impute = []

for elem in all_numerics_wo_encoded:
    if elem not in numericals_zero_impute:
        numericals_mean_impute.append(elem)

In [614]:
print(X_2.shape[1])
print(len(all_numerics))
print(len(all_cats))
print(len(encoded_columns))
print(len(all_numerics_wo_encoded))
print(len(numericals_mean_impute))
print(len(numericals_zero_impute))

223
157
7
68
89
82
7


In [615]:
num_zero_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", MinMaxScaler())
])

num_mean_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

cat_tr = OneHotEncoder(handle_unknown='ignore', )

In [639]:
preprocessor = ColumnTransformer(
    [("numerics_zero_imputing", num_zero_tr, numericals_zero_impute),
     ("numerics_mean_imputing", num_mean_tr, numericals_mean_impute),
     ("cat_tr", cat_tr, all_cats)],
remainder="passthrough")

pipe_2 = Pipeline([
    ("preprocessing", preprocessor), 
    ("regressor", Lasso())])

pipe_2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics_zero_imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['gk_diving', 'gk_handling',
                                                   'gk_kicking', 'gk_reflexes',
                                                   'gk_speed', 'gk_positioning',
                                                   'release_clause_eur']),
                                                 ('numerics_mean_imputing',...
                                                   'skill_dribbling',
                                                   'skill_curve',
                                                   'skill_fk_accuracy',
                                                   'skill_long_passing',
                                                   'skill_ball_control',
                                                   'movement_acceleration',
                                                   'movement_sprint_speed',
                                                   'movement_agility', ...]),
                                                 ('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['preferred_foot',
                                                   'work_rate', 'body_type',
                                                   'real_face',
                                                   'nationality_cleaned',
                                                   'league_name_cleaned',
                                                   'position_cluster'])])),
                ('regressor', Lasso())])

In [635]:
X_2 = df_train.drop(columns="fee_cleaned")
y_2 = df_train[["fee_cleaned"]]

In [636]:
cross_val_score(pipe_2, X_2, y_2, cv=5, scoring="r2").mean()

0.006457387480565413

In [640]:
pipe_2.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('numerics_zero_imputing',
                                    Pipeline(steps=[('imputer',
                                                     SimpleImputer(fill_value=0,
                                                                   strategy='constant')),
                                                    ('scaler', MinMaxScaler())]),
                                    ['gk_diving', 'gk_handling', 'gk_kicking',
                                     'gk_reflexes', 'gk_speed', 'gk_positioning',
                                     'release_clause_eur']),
                                   ('numerics_mean_imputing',
                                    Pipeline(steps=[('imputer', Simple...
                                     'attacking_volleys', 'skill_dribbling',
                                     'skill_curve', 'skill_fk_accuracy',
                     

## Grid search

In [644]:
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

In [647]:
from sklearn.model_selection import GridSearchCV

In [645]:
param_grid={
#     'columntransformer__num_tr__scaler': [RobustScaler()],
    'regressor': [
        Ridge(), 
        #LinearRegression(), 
        XGBRegressor(), 
        CatBoostRegressor(), 
        SGDRegressor(), 
        #KernelRidge(), 
        #ElasticNet(),
        #BayesianRidge(), 
        #GradientBoostingRegressor(),
        #SVR()
    ]}

In [648]:
grid_search = GridSearchCV(
    pipe_2, 
    param_grid=param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1)

grid_search.fit(X_2, y_2)
print(grid_search.best_params_)
print(grid_search.best_score_)

Learning rate set to 0.047291
0:	learn: 11.1012040	total: 329ms	remaining: 5m 28s
1:	learn: 10.9410770	total: 423ms	remaining: 3m 30s
2:	learn: 10.7947478	total: 606ms	remaining: 3m 21s
3:	learn: 10.6542670	total: 686ms	remaining: 2m 50s
4:	learn: 10.5179182	total: 900ms	remaining: 2m 59s
5:	learn: 10.3766421	total: 1.08s	remaining: 2m 59s
6:	learn: 10.2529840	total: 1.14s	remaining: 2m 42s
7:	learn: 10.1228661	total: 1.29s	remaining: 2m 39s
8:	learn: 10.0150990	total: 1.36s	remaining: 2m 30s
9:	learn: 9.9077950	total: 1.44s	remaining: 2m 22s
10:	learn: 9.7953198	total: 1.52s	remaining: 2m 17s
11:	learn: 9.7098350	total: 1.59s	remaining: 2m 10s
12:	learn: 9.6241493	total: 1.7s	remaining: 2m 8s
13:	learn: 9.5376101	total: 1.77s	remaining: 2m 5s
14:	learn: 9.4544618	total: 1.85s	remaining: 2m 1s
15:	learn: 9.4274859	total: 1.97s	remaining: 2m 1s
16:	learn: 9.3960651	total: 2.07s	remaining: 1m 59s
17:	learn: 9.3367823	total: 2.12s	remaining: 1m 55s
18:	learn: 9.2587035	total: 2.24s	remain

Learning rate set to 0.047291
0:	learn: 10.9483260	total: 249ms	remaining: 4m 8s
1:	learn: 10.8163032	total: 349ms	remaining: 2m 54s
2:	learn: 10.6739505	total: 481ms	remaining: 2m 39s
3:	learn: 10.5352768	total: 600ms	remaining: 2m 29s
4:	learn: 10.4015343	total: 700ms	remaining: 2m 19s
5:	learn: 10.2803938	total: 779ms	remaining: 2m 9s
6:	learn: 10.1555518	total: 883ms	remaining: 2m 5s
7:	learn: 10.0530319	total: 1.15s	remaining: 2m 22s
8:	learn: 9.9374007	total: 1.35s	remaining: 2m 28s
9:	learn: 9.8384752	total: 1.41s	remaining: 2m 19s
10:	learn: 9.7352872	total: 1.47s	remaining: 2m 12s
11:	learn: 9.6436710	total: 1.73s	remaining: 2m 22s
12:	learn: 9.5513131	total: 1.85s	remaining: 2m 20s
13:	learn: 9.4612458	total: 1.91s	remaining: 2m 14s
14:	learn: 9.3688949	total: 1.96s	remaining: 2m 8s
15:	learn: 9.2833407	total: 2.08s	remaining: 2m 7s
16:	learn: 9.2027505	total: 2.21s	remaining: 2m 7s
17:	learn: 9.1383080	total: 2.31s	remaining: 2m 5s
18:	learn: 9.0790668	total: 2.4s	remaining:

157:	learn: 6.5271467	total: 18.4s	remaining: 1m 38s
158:	learn: 6.5195498	total: 18.6s	remaining: 1m 38s
159:	learn: 6.5041395	total: 18.6s	remaining: 1m 37s
160:	learn: 6.4962734	total: 18.7s	remaining: 1m 37s
161:	learn: 6.4889154	total: 18.8s	remaining: 1m 37s
162:	learn: 6.4852552	total: 18.9s	remaining: 1m 37s
163:	learn: 6.4807291	total: 18.9s	remaining: 1m 36s
164:	learn: 6.4766382	total: 19s	remaining: 1m 36s
165:	learn: 6.4657047	total: 19.1s	remaining: 1m 35s
166:	learn: 6.4569421	total: 19.2s	remaining: 1m 35s
167:	learn: 6.4446299	total: 19.2s	remaining: 1m 35s
168:	learn: 6.4418525	total: 19.4s	remaining: 1m 35s
169:	learn: 6.4346420	total: 19.5s	remaining: 1m 35s
170:	learn: 6.4314310	total: 19.6s	remaining: 1m 34s
171:	learn: 6.4156711	total: 19.6s	remaining: 1m 34s
172:	learn: 6.4080908	total: 19.7s	remaining: 1m 34s
173:	learn: 6.4063494	total: 19.8s	remaining: 1m 34s
174:	learn: 6.4033159	total: 19.9s	remaining: 1m 33s
175:	learn: 6.3973706	total: 20s	remaining: 1m 3

312:	learn: 5.0770102	total: 30.9s	remaining: 1m 7s
313:	learn: 5.0672326	total: 31.2s	remaining: 1m 8s
314:	learn: 5.0624307	total: 31.3s	remaining: 1m 8s
315:	learn: 5.0599435	total: 31.4s	remaining: 1m 7s
316:	learn: 5.0531613	total: 31.5s	remaining: 1m 7s
317:	learn: 5.0473840	total: 31.6s	remaining: 1m 7s
318:	learn: 5.0406240	total: 31.7s	remaining: 1m 7s
319:	learn: 5.0363398	total: 31.7s	remaining: 1m 7s
320:	learn: 5.0222877	total: 31.8s	remaining: 1m 7s
321:	learn: 5.0136471	total: 31.9s	remaining: 1m 7s
322:	learn: 5.0076048	total: 32s	remaining: 1m 7s
323:	learn: 5.0036713	total: 32.1s	remaining: 1m 6s
324:	learn: 4.9896573	total: 32.1s	remaining: 1m 6s
325:	learn: 4.9782725	total: 32.3s	remaining: 1m 6s
326:	learn: 4.9686492	total: 32.4s	remaining: 1m 6s
327:	learn: 4.9625808	total: 32.4s	remaining: 1m 6s
328:	learn: 4.9519820	total: 32.5s	remaining: 1m 6s
329:	learn: 4.9460191	total: 32.5s	remaining: 1m 6s
330:	learn: 4.9409080	total: 32.6s	remaining: 1m 5s
331:	learn: 4.

312:	learn: 5.4352877	total: 32.9s	remaining: 1m 12s
313:	learn: 5.4238317	total: 32.9s	remaining: 1m 11s
314:	learn: 5.4204039	total: 33s	remaining: 1m 11s
315:	learn: 5.4186918	total: 33.1s	remaining: 1m 11s
316:	learn: 5.4040255	total: 33.2s	remaining: 1m 11s
317:	learn: 5.3951101	total: 33.3s	remaining: 1m 11s
318:	learn: 5.3850471	total: 33.4s	remaining: 1m 11s
319:	learn: 5.3745685	total: 33.5s	remaining: 1m 11s
320:	learn: 5.3728999	total: 33.5s	remaining: 1m 10s
321:	learn: 5.3719046	total: 33.6s	remaining: 1m 10s
322:	learn: 5.3600169	total: 33.7s	remaining: 1m 10s
323:	learn: 5.3460056	total: 33.7s	remaining: 1m 10s
324:	learn: 5.3376115	total: 33.8s	remaining: 1m 10s
325:	learn: 5.3286898	total: 33.8s	remaining: 1m 9s
326:	learn: 5.3222406	total: 33.9s	remaining: 1m 9s
327:	learn: 5.3093842	total: 34s	remaining: 1m 9s
328:	learn: 5.3031665	total: 34s	remaining: 1m 9s
329:	learn: 5.2925363	total: 34.1s	remaining: 1m 9s
330:	learn: 5.2916367	total: 34.1s	remaining: 1m 8s
331:	

...

536:	learn: 4.3072794	total: 3.31s	remaining: 2.85s
537:	learn: 4.2981005	total: 3.31s	remaining: 2.85s
538:	learn: 4.2948074	total: 3.32s	remaining: 2.84s
539:	learn: 4.2944375	total: 3.33s	remaining: 2.83s
540:	learn: 4.2895859	total: 3.33s	remaining: 2.83s
541:	learn: 4.2841665	total: 3.34s	remaining: 2.82s
542:	learn: 4.2778864	total: 3.35s	remaining: 2.82s
543:	learn: 4.2740333	total: 3.36s	remaining: 2.82s
544:	learn: 4.2684472	total: 3.37s	remaining: 2.81s
545:	learn: 4.2629814	total: 3.37s	remaining: 2.8s
546:	learn: 4.2627468	total: 3.38s	remaining: 2.8s
547:	learn: 4.2583017	total: 3.38s	remaining: 2.79s
548:	learn: 4.2534701	total: 3.39s	remaining: 2.78s
549:	learn: 4.2471245	total: 3.4s	remaining: 2.78s
550:	learn: 4.2428893	total: 3.4s	remaining: 2.77s
551:	learn: 4.2380425	total: 3.41s	remaining: 2.77s
552:	learn: 4.2330835	total: 3.42s	remaining: 2.76s
553:	learn: 4.2299155	total: 3.42s	remaining: 2.76s
554:	learn: 4.2239199	total: 3.43s	remaining: 2.75s
555:	learn: 4.21

720:	learn: 3.6228402	total: 4.48s	remaining: 1.73s
721:	learn: 3.6220521	total: 4.48s	remaining: 1.73s
722:	learn: 3.6185192	total: 4.49s	remaining: 1.72s
723:	learn: 3.6182807	total: 4.49s	remaining: 1.71s
724:	learn: 3.6143002	total: 4.5s	remaining: 1.71s
725:	learn: 3.6124253	total: 4.5s	remaining: 1.7s
726:	learn: 3.6080044	total: 4.51s	remaining: 1.69s
727:	learn: 3.6045165	total: 4.51s	remaining: 1.69s
728:	learn: 3.5991659	total: 4.52s	remaining: 1.68s
729:	learn: 3.5951386	total: 4.53s	remaining: 1.68s
730:	learn: 3.5926862	total: 4.54s	remaining: 1.67s
731:	learn: 3.5900358	total: 4.54s	remaining: 1.66s
732:	learn: 3.5862087	total: 4.55s	remaining: 1.66s
733:	learn: 3.5817626	total: 4.56s	remaining: 1.65s
734:	learn: 3.5809345	total: 4.56s	remaining: 1.64s
735:	learn: 3.5784116	total: 4.57s	remaining: 1.64s
736:	learn: 3.5732682	total: 4.57s	remaining: 1.63s
737:	learn: 3.5678277	total: 4.58s	remaining: 1.62s
738:	learn: 3.5642177	total: 4.58s	remaining: 1.62s
739:	learn: 3.5

882:	learn: 3.1758646	total: 5.44s	remaining: 721ms
883:	learn: 3.1721104	total: 5.44s	remaining: 714ms
884:	learn: 3.1693813	total: 5.45s	remaining: 708ms
885:	learn: 3.1657962	total: 5.46s	remaining: 702ms
886:	learn: 3.1637444	total: 5.46s	remaining: 696ms
887:	learn: 3.1588064	total: 5.47s	remaining: 689ms
888:	learn: 3.1561524	total: 5.47s	remaining: 683ms
889:	learn: 3.1533377	total: 5.47s	remaining: 677ms
890:	learn: 3.1514124	total: 5.48s	remaining: 671ms
891:	learn: 3.1497604	total: 5.49s	remaining: 664ms
892:	learn: 3.1483761	total: 5.49s	remaining: 658ms
893:	learn: 3.1466604	total: 5.5s	remaining: 652ms
894:	learn: 3.1450055	total: 5.5s	remaining: 646ms
895:	learn: 3.1426207	total: 5.51s	remaining: 639ms
896:	learn: 3.1400275	total: 5.51s	remaining: 633ms
897:	learn: 3.1371581	total: 5.52s	remaining: 627ms
898:	learn: 3.1340332	total: 5.53s	remaining: 621ms
899:	learn: 3.1322877	total: 5.53s	remaining: 615ms
900:	learn: 3.1288119	total: 5.54s	remaining: 609ms
901:	learn: 3.

/home/alexpins/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/alexpins/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/alexpins/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/alexpins/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/utils/vali

890:	learn: 2.8358527	total: 1m 1s	remaining: 7.47s
891:	learn: 2.8334169	total: 1m 1s	remaining: 7.4s
892:	learn: 2.8304420	total: 1m 1s	remaining: 7.34s
893:	learn: 2.8271004	total: 1m 1s	remaining: 7.26s
894:	learn: 2.8235591	total: 1m 1s	remaining: 7.2s
895:	learn: 2.8201179	total: 1m 1s	remaining: 7.13s
896:	learn: 2.8178994	total: 1m 1s	remaining: 7.07s
897:	learn: 2.8144318	total: 1m 1s	remaining: 7s
898:	learn: 2.8107130	total: 1m 1s	remaining: 6.94s
899:	learn: 2.8099169	total: 1m 1s	remaining: 6.87s
900:	learn: 2.8079969	total: 1m 1s	remaining: 6.8s
901:	learn: 2.8042558	total: 1m 1s	remaining: 6.73s
902:	learn: 2.8017856	total: 1m 2s	remaining: 6.66s
903:	learn: 2.7993298	total: 1m 2s	remaining: 6.59s
904:	learn: 2.7966056	total: 1m 2s	remaining: 6.52s
905:	learn: 2.7963059	total: 1m 2s	remaining: 6.45s
906:	learn: 2.7919503	total: 1m 2s	remaining: 6.38s
907:	learn: 2.7890431	total: 1m 2s	remaining: 6.31s
908:	learn: 2.7864428	total: 1m 2s	remaining: 6.25s
909:	learn: 2.7846